# Tesla Stock Price Prediction - LSTM Model

## Objective
- Build LSTM models for 1-day, 5-day, and 10-day predictions
- Use GridSearchCV (via Keras Tuner) for hyperparameter tuning
- Tune: LSTM units, dropout rate, learning rate

### Setup Notes
This notebook uses **PyTorch** for better Windows compatibility. Install torch if not already available.

In [2]:
import subprocess
import sys
print("Installing PyTorch (better Windows support)...")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "torch", "--index-url", "https://download.pytorch.org/whl/cpu"], timeout=300)
    print("PyTorch installed successfully")
except Exception as e:
    print(f"PyTorch install attempted: {e}")

Installing PyTorch (better Windows support)...
PyTorch installed successfully


In [3]:
import numpy as np
from pathlib import Path
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error

PROJECT_ROOT = Path('..').resolve()
DATA_PATH = PROJECT_ROOT / 'data' / 'processed'
MODELS_PATH = PROJECT_ROOT / 'models'
MODELS_PATH.mkdir(exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, output_size=1, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        lstm_out, (h_n, c_n) = self.lstm(x)
        lstm_out = self.dropout(lstm_out)
        predictions = self.fc(lstm_out[:, -1, :])
        return predictions

def build_lstm_model(units=64, dropout=0.2):
    return LSTMModel(input_size=1, hidden_size=units, output_size=1, dropout=dropout)

In [5]:
# Simple grid search for hyperparameter tuning on 1-day data
from torch.utils.data import TensorDataset, DataLoader

X_train = np.load(DATA_PATH / 'X_train_1d.npy')
y_train = np.load(DATA_PATH / 'y_train_1d.npy')
X_test = np.load(DATA_PATH / 'X_test_1d.npy')
y_test = np.load(DATA_PATH / 'y_test_1d.npy')

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1).to(device)

# Grid search parameters
lstm_units_options = [32, 64, 96]
dropout_options = [0.1, 0.2, 0.3]
lr_options = [0.001, 0.0005]

best_val_loss = float('inf')
best_params = {}

print("Hyperparameter tuning (grid search on 1-day data)...")
for units in lstm_units_options:
    for dropout in dropout_options:
        for lr in lr_options:
            model = build_lstm_model(units=units, dropout=dropout).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            criterion = nn.MSELoss()
            
            train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
            
            # Train for limited epochs to find best params
            for epoch in range(20):
                model.train()
                total_loss = 0
                for X_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    loss.backward()
                    optimizer.step()
                    total_loss += loss.item()
                
                # Validation
                model.eval()
                with torch.no_grad():
                    val_outputs = model(X_train_tensor)
                    val_loss = criterion(val_outputs, y_train_tensor).item()
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_params = {'units': units, 'dropout': dropout, 'lr': lr}
                print(f"  Units: {units}, Dropout: {dropout}, LR: {lr} → Val Loss: {val_loss:.6f}")

print(f"\nBest params: LSTM units={best_params['units']}, Dropout={best_params['dropout']}, LR={best_params['lr']}")

Hyperparameter tuning (grid search on 1-day data)...
  Units: 32, Dropout: 0.1, LR: 0.001 → Val Loss: 0.000190
  Units: 32, Dropout: 0.1, LR: 0.0005 → Val Loss: 0.000167
  Units: 32, Dropout: 0.2, LR: 0.001 → Val Loss: 0.000136
  Units: 64, Dropout: 0.1, LR: 0.001 → Val Loss: 0.000118
  Units: 64, Dropout: 0.2, LR: 0.001 → Val Loss: 0.000110

Best params: LSTM units=64, Dropout=0.2, LR=0.001


In [6]:
# Train LSTM models for all horizons with best tuned params
horizons = [1, 5, 10]
results = []

for horizon in horizons:
    print(f"\n=== Training {horizon}-day LSTM ===")
    
    # Load data
    X_train = np.load(DATA_PATH / f'X_train_{horizon}d.npy')
    y_train = np.load(DATA_PATH / f'y_train_{horizon}d.npy')
    X_test = np.load(DATA_PATH / f'X_test_{horizon}d.npy')
    y_test = np.load(DATA_PATH / f'y_test_{horizon}d.npy')
    
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train).to(device)
    y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1).to(device)
    X_test_tensor = torch.FloatTensor(X_test).to(device)
    y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # Build model with best params
    model = build_lstm_model(units=best_params['units'], dropout=best_params['dropout']).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'])
    criterion = nn.MSELoss()
    
    # Training loop with early stopping
    epochs = 100
    patience = 10
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_train_tensor)
            val_loss = criterion(val_outputs, y_train_tensor).item()
        
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), MODELS_PATH / f'lstm_{horizon}day_best.pt')
        else:
            patience_counter += 1
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_loss:.6f}, Val Loss: {val_loss:.6f}")
        
        # Early stopping
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break
    
    # Evaluate on test set
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor).cpu().numpy()
    
    mse = mean_squared_error(y_test, y_pred)
    results.append({'horizon': horizon, 'mse': mse})
    print(f"{horizon}-day LSTM MSE: {mse:.6f}\n")

print("=" * 50)
print("Final Results:", results)
print("=" * 50)


=== Training 1-day LSTM ===
Epoch 10/100, Train Loss: 0.000398, Val Loss: 0.000152
Epoch 20/100, Train Loss: 0.000363, Val Loss: 0.000121
Epoch 30/100, Train Loss: 0.000258, Val Loss: 0.000115
Epoch 40/100, Train Loss: 0.000203, Val Loss: 0.000092
Epoch 50/100, Train Loss: 0.000193, Val Loss: 0.000081
Epoch 60/100, Train Loss: 0.000191, Val Loss: 0.000082
Epoch 70/100, Train Loss: 0.000171, Val Loss: 0.000066
Epoch 80/100, Train Loss: 0.000165, Val Loss: 0.000071
Epoch 90/100, Train Loss: 0.000162, Val Loss: 0.000067
Epoch 100/100, Train Loss: 0.000152, Val Loss: 0.000077
1-day LSTM MSE: 0.000330


=== Training 5-day LSTM ===
Epoch 10/100, Train Loss: 0.000734, Val Loss: 0.000299
Epoch 20/100, Train Loss: 0.000564, Val Loss: 0.000282
Epoch 30/100, Train Loss: 0.000501, Val Loss: 0.000281
Epoch 40/100, Train Loss: 0.000430, Val Loss: 0.000268
Epoch 50/100, Train Loss: 0.000389, Val Loss: 0.000261
Epoch 60/100, Train Loss: 0.000392, Val Loss: 0.000252
Epoch 70/100, Train Loss: 0.000373,